In [1]:
import numpy as np
import random

In [4]:
import random

# Definimos funciones primitivas para la PG
def add(x, y): return x + y
def sub(x, y): return x - y
def mul(x, y): return x * y
def div(x, y): return x / y if y != 0 else 1

# Generamos una población inicial de árboles de fórmulas
def generar_poblacion(tamano):
    funciones = [add, sub, mul, div]
    terminales = ["peso", "granos", 1, 2, 3]
    poblacion = []
    for _ in range(tamano):
        arbol = [random.choice(funciones), random.choice(terminales), random.choice(terminales)]
        poblacion.append(arbol)
    return poblacion

# Evaluamos una fórmula en un contexto
def evaluar(arbol, contexto):
    if isinstance(arbol, list) and callable(arbol[0]):  # Aseguramos que sea una lista con una función
        return arbol[0](evaluar(arbol[1], contexto), evaluar(arbol[2], contexto))
    else:  # Es un terminal
        return contexto[arbol] if arbol in contexto else arbol

# Fitness: qué tan bien una fórmula optimiza la velocidad
def calcular_fitness(arbol, datos):
    error_total = 0
    for dato in datos:
        prediccion = evaluar(arbol, dato)
        error_total += abs(prediccion - dato["velocidad_ideal"])
    return 1 / (1 + error_total)

# Evolucionamos la población
def evolucionar(poblacion, datos, generaciones=10):
    funciones = [add, sub, mul, div]
    terminales = ["peso", "granos", 1, 2, 3]

    for _ in range(generaciones):
        fitness = [calcular_fitness(arbol, datos) for arbol in poblacion]
        seleccionados = random.choices(poblacion, weights=fitness, k=len(poblacion))
        nueva_poblacion = []

        for i in range(0, len(seleccionados), 2):
            # Cruzamiento
            padre1, padre2 = seleccionados[i], seleccionados[i+1]
            hijo1 = [padre1[0], padre2[1], padre1[2]]
            hijo2 = [padre2[0], padre1[1], padre2[2]]
            nueva_poblacion += [hijo1, hijo2]

        # Mutación
        for arbol in nueva_poblacion:
            if random.random() < 0.1:  # Probabilidad de mutar
                idx = random.randint(0, 2)  # Elegimos un índice para mutar
                if idx == 0:  # Si es la función, mutamos la función
                    arbol[idx] = random.choice(funciones)
                else:  # Si es un terminal, mutamos el terminal
                    arbol[idx] = random.choice(terminales)

        poblacion = nueva_poblacion

    return max(poblacion, key=lambda a: calcular_fitness(a, datos))

# Ejemplo de datos de entrenamiento
datos_entrenamiento = [
    {"peso": 10, "granos": 5, "velocidad_ideal": 7},
    {"peso": 20, "granos": 10, "velocidad_ideal": 15},
    # Agrega más datos aquí
]

# Ejecución
poblacion_inicial = generar_poblacion(10)
mejor_solucion = evolucionar(poblacion_inicial, datos_entrenamiento)
print("Mejor fórmula encontrada:", mejor_solucion)


Mejor fórmula encontrada: [<function add at 0x7974fee63e20>, 2, 'granos']
